In [1]:
import os
import fnmatch
import cv2
import numpy as np
import json
import sklearn.model_selection
from keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

from data_generator import DataGenerator

from model_crnn import CRNN

from tensorflow.keras.utils import Sequence
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior
#ignore warnings in the output
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13254106414103033721
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3149044121
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4764495037773098239
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:3a:00.0, compute capability: 6.1"
]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050, pci bus id: 0000:3a:00.0, compute capability: 6.1

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16156643316208265129
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3149044121
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14407566107356059942
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:3a:00.0, compute capability: 6.1"
]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050, pci bus id: 0000:3a:00

In [2]:
 with open('./label_word_num.json',encoding='UTF8') as f:
    label_word_num = json.load(f)
with open('./label_num_word.json',encoding='UTF8') as f:
    label_num_word = json.load(f)
with open('./label_text.json',encoding='UTF8') as f:
    label_text = json.load(f)
with open('./img_list.json',encoding='UTF8') as f:
    img_list = json.load(f)

In [3]:
model_train, model_pred = CRNN()

model_train.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
 
filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [4]:
train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(img_list, label_text, test_size=0.2, random_state=1)
train_x, val_x, train_y, val_y = sklearn.model_selection.train_test_split(train_x, train_y, test_size=0.2, random_state=1)

In [5]:
print(len(train_x))
print(len(train_y))
print(len(val_x))
print(len(val_y))
print(len(test_x))
print(len(test_y))
print(train_y[0])

72329
72329
18083
18083
22604
22604
[327, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
train_data = DataGenerator(train_x, train_y, 128)
val_data = DataGenerator(val_x, val_y,128)
test_data = DataGenerator(test_x, test_y, 128)

train_steps = len(train_x) // 128
val_steps = len(val_x) // 128
epochs = 50

In [7]:
model_train.fit_generator(generator=train_data, # batch_size here?
                    steps_per_epoch=train_steps,
                    epochs=1,
                    validation_data=val_data, # batch_size here?
                    validation_steps=train_steps,   
                    callbacks = callbacks_list,
                    initial_epoch=0)

300/565 [==============>...............] - ETA: 11:13 - loss: inf

ValueError: Input arrays should have the same number of samples as target arrays. Found 9 input samples and 128 target samples.